In [1]:
!scrapy startproject gmarket

Error: scrapy.cfg already exists in /Users/yeonghwanchoi/Documents/git/crawling/gmarket_scrapy/gmarket


In [2]:
!tree gmarket

gmarket
├── gmarket
│   ├── __init__.py
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       └── sipder.py
├── gmarket.items.py
└── scrapy.cfg

2 directories, 9 files


In [3]:
import scrapy, requests
from scrapy.http import TextResponse

In [4]:
response = requests.get("http://corners.gmarket.co.kr/Bestsellers")
dom = TextResponse(response.url, body=response.text, encoding="utf-8")

In [5]:
links = dom.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li/div[1]/a/@href').extract()
len(links), links[0]

(200,
 'http://item.gmarket.co.kr/Item?goodscode=2012381280&ver=637498390693961234')

In [6]:
link = links[5]
print(link)

http://item.gmarket.co.kr/Item?goodscode=1964113036&ver=637498390693961234


In [7]:
response = requests.get(link)
dom = TextResponse(response.url, body=response.text, encoding="utf-8")

In [8]:
title = dom.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract().strip()
s_price = dom.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
try:
    o_price = dom.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/span/text()')[0].extract()
except:
    o_price = s_price
title, s_price, o_price

('자연 모짜렐라 슈레드치즈 1kg  /피자치즈/대용량', '10,500', '21,000')

In [9]:
!tree gmarket

gmarket
├── gmarket
│   ├── __init__.py
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       └── sipder.py
├── gmarket.items.py
└── scrapy.cfg

2 directories, 9 files


In [ ]:
# %load gmarket/gmarket/items.py
import scrapy

class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    s_price = scrapy.Field()
    o_price = scrapy.Field()
    link = scrapy.Field()


In [11]:
%%writefile gmarket/gmarket/items.py
import scrapy

class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    s_price = scrapy.Field()
    o_price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [12]:
%%writefile gmarket/gmarket/spiders/sipder.py
import scrapy
from gmarket.items import GmarketItem

class Spider(scrapy.Spider):
    name = "GmarketBest"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li/div[1]/a/@href').extract()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract().strip()
        item["s_price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        try:
            item["o_price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/span/text()')[0].extract()
        except:
            item["o_price"] = item["s_price"]
        item["link"] = response.url
        yield item        

Overwriting gmarket/gmarket/spiders/sipder.py


In [13]:
!ls

gmarket       gmarket.ipynb run.sh


In [14]:
!tree

.
├── gmarket
│   ├── gmarket
│   │   ├── __init__.py
│   │   ├── items.py
│   │   ├── middlewares.py
│   │   ├── pipelines.py
│   │   ├── settings.py
│   │   └── spiders
│   │       ├── __init__.py
│   │       └── sipder.py
│   ├── gmarket.items.py
│   └── scrapy.cfg
├── gmarket.ipynb
└── run.sh

3 directories, 11 files


In [15]:
%%writefile run.sh
cd gmarket
rm gmarket.csv
scrapy crawl GmarketBest -o gmarket.csv

Overwriting run.sh


In [16]:
%ls gmarket

gmarket/          gmarket.items.py  scrapy.cfg


In [17]:
!source run.sh

rm: gmarket.csv: No such file or directory
2021-02-25 08:38:04 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: gmarket)
2021-02-25 08:38:04 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  4 2020, 02:22:02) - [Clang 10.0.0 ], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform macOS-10.16-x86_64-i386-64bit
2021-02-25 08:38:04 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-02-25 08:38:04 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'gmarket',
 'NEWSPIDER_MODULE': 'gmarket.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['gmarket.spiders']}
2021-02-25 08:38:04 [scrapy.extensions.telnet] INFO: Telnet Password: 4ae4239d712a6cc4
2021-02-25 08:38:04 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.m

2021-02-25 08:38:06 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=2030368041&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=2030368041&ver=637498390848834455',
 'o_price': '54,000',
 's_price': '43,200',
 'title': '컬럼비아  남 여 경량 바람막이 자켓 C12-XO0338.WR0642'}
2021-02-25 08:38:06 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1789408830&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1789408830&ver=637498390848834455',
 'o_price': '23,500',
 's_price': '22,330',
 'title': '갤러리아  꿀고구마 베니하루카 5kg (특상품)'}
2021-02-25 08:38:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=2036757780&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:06 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1232319673&ver=637498390848834455>
{'link': 'http://item.

2021-02-25 08:38:07 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=2035718461&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=2035718461&ver=637498390848834455',
 'o_price': '20,800',
 's_price': '13,900',
 'title': '바다내음 통통 활멍게 2Kg / 20% 쿠폰가 10770원'}
2021-02-25 08:38:07 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1975776752&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1975776752&ver=637498390848834455',
 'o_price': '42,900',
 's_price': '12,900',
 'title': '쇼인더룸 9900특가 봄신상/티셔츠/맨투맨/팬츠/원피스'}
2021-02-25 08:38:07 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1754998124&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1754998124&ver=637498390848834455',
 'o_price': '32,900',
 's_price': '32,900',
 'title': '크리넥스 3겹 데코소프트 33MX24롤 X2팩화장지/휴지'}
2021-02-25 08:38:07 [scrapy.core.scraper] D

2021-02-25 08:38:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1614191721&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1614191721&ver=637498390848834455',
 'o_price': '30,000',
 's_price': '20,500',
 'title': '진라면 매운맛 40봉'}
2021-02-25 08:38:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=806338693&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=806338693&ver=637498390848834455',
 'o_price': '33,000',
 's_price': '9,900',
 'title': '바브키즈 쿠폰할인 10%~ 봄신상 남여공용 아이템'}
2021-02-25 08:38:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1649424410&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1297944657&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)

2021-02-25 08:38:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1854481327&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1559158003&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1559158003&ver=637498390848834455',
 'o_price': '15,900',
 's_price': '12,500',
 'title': '하늘보리 1.5L x 12입'}
2021-02-25 08:38:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1854481327&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1854481327&ver=637498390848834455',
 'o_price': '10,990',
 's_price': '10,720',
 'title': '팔도진미_강원도춘천식닭갈비_1kg'}
2021-02-25 08:38:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1998577955&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2

...

2021-02-25 08:38:18 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1845754094&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1845754094&ver=637498390848834455',
 'o_price': '33,900',
 's_price': '33,900',
 'title': '하기스 맥스드라이팬티 4단계 공용 38PX2팩'}
2021-02-25 08:38:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1930999301&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1556120394&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1516542332&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?good

2021-02-25 08:38:19 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1884943804&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1884943804&ver=637498390848834455',
 'o_price': '30,900',
 's_price': '29,200',
 'title': 'CJ제일제당  비비고 만두 총 10봉 (왕교자/김치왕교'}
2021-02-25 08:38:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1909861349&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:20 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1909861349&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1909861349&ver=637498390848834455',
 'o_price': '29,800',
 's_price': '14,900',
 'title': '치약 뉴후레쉬 알파치약 170g 10+10개'}
2021-02-25 08:38:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1693433901&ver=637498390848834455> (referer: http://corners.gmarket

2021-02-25 08:38:21 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1108678685&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1108678685&ver=637498390848834455',
 'o_price': '23,000',
 's_price': '6,900',
 'title': '봄신상 밴딩팬츠/슬랙스/부츠컷/통바지/빅사이즈~4XL'}
2021-02-25 08:38:21 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1926996726&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1926996726&ver=637498390848834455',
 'o_price': '19,800',
 's_price': '19,210',
 'title': '1+1+1 아이몽땅버블 폼폼휘핑클렌저250ml 6종'}
2021-02-25 08:38:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1499998778&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=2029892424&ver=637498390848834455> (referer: http://corners.gmark

2021-02-25 08:38:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1920769262&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=2031953229&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:23 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=2041894131&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=2041894131&ver=637498390848834455',
 'o_price': '18,900',
 's_price': '18,720',
 'title': '아가방 봄 에단맨투맨상하(2Color)_01N553001'}
2021-02-25 08:38:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=2041431816&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:23 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Ite

2021-02-25 08:38:24 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=2035884493&ver=637498390848834455>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=2035884493&ver=637498390848834455',
 'o_price': '33,920',
 's_price': '33,920',
 'title': '풀무원 얇은피만두 6종 8봉 노엣지피자 5판 골라담기'}
2021-02-25 08:38:25 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1942784034&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:25 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1964113036&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:25 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=2029885837&ver=637498390848834455> (referer: http://corners.gmarket.co.kr/Bestsellers)
2021-02-25 08:38:25 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?

In [18]:
df = pd.read_csv("gmarket/gmarket.csv")

In [19]:
df

,link,o_price,s_price,title
0,http://item.gmarket.co.kr/Item?goodscode=14161...,"28,300","18,900",강원도 곤드레 비벼요 고소한맛 85g X 10봉 (10인분)
1,http://item.gmarket.co.kr/Item?goodscode=19644...,"21,000","17,900",켈로그 그래놀라 3종(블루베리+크렌베리+현미)
2,http://item.gmarket.co.kr/Item?goodscode=16783...,"13,300","8,900",여수사나이 총각김치/김치/국내산 1.5kg/당일생산
3,http://item.gmarket.co.kr/Item?goodscode=20123...,"10,900","10,900",무안 박진우님의 고구마 2kg 특상중 꿀고구마
4,http://item.gmarket.co.kr/Item?goodscode=19711...,"44,000","30,900",모데즈 유모차라이너 (디자인선택)
...,...,...,...,...
195,http://item.gmarket.co.kr/Item?goodscode=19641...,"21,000","10,500",자연 모짜렐라 슈레드치즈 1kg /피자치즈/대용량
196,http://item.gmarket.co.kr/Item?goodscode=20298...,"27,900","13,900",춘천 식 숯불 닭갈비 750g x 2봉
197,http://item.gmarket.co.kr/Item?goodscode=90063...,"33,000","9,900",봄신상특가12%+베페추가쿠폰 상하복/원피스/가디건
198,http://item.gmarket.co.kr/Item?goodscode=18133...,"32,900","32,900",다향오리 훈제슬라이스 200gx10팩
